In [1]:
from mohan.Similarity import Similarity
from pymongo import MongoClient
from joblib import Parallel, delayed

In [2]:
es_handler = Similarity("kahi")

In [3]:
print(es_handler.es.cat.indices())

yellow open openalex_parsed tihIWltzTUOlEQ_3MFNluA 1 1 203800 0      1gb      1gb
yellow open kahi            freWYijvTKuNolyDNVruiA 1 1      0 0     225b     225b
yellow open openalex_raw    9ThyTMV8TzyMNZtRidOkRA 1 1 203800 0 1023.1mb 1023.1mb
yellow open kahi_es         cXkaLQcITJOvn2YZe7Ibtg 1 1    345 0  247.3kb  247.3kb
yellow open oa_dfr          8M6l0GgGS1yznRLqLTuaXQ 1 1 203800 0      1gb      1gb
yellow open kahi_all        X_U9yff9RJ2VrvzwEyk4zQ 1 1      0 0     225b     225b



In [4]:
#es_handler.delete_index("kahi")

In [5]:
kahi_data = list(MongoClient()["kahi_all"]["works"].find())

In [6]:

#este código permite carga kahi de los artículos con DOI en elastic search para "entrenar" el indice,
# la similaridad luego se hace con los artículos sin DOI
def insert_work(entry):
    work = {}
    if len(entry["titles"]) >0:
        work["title"] = entry["titles"][0]["title"]
    else:
        return entry
    work["source"] = entry["source"]["name"]
    work["year"] = entry["year_published"]
    work["volume"] = entry["bibliographic_info"]["volume"] if "volume" in entry["bibliographic_info"].keys() else ""
    work["issue"] = entry["bibliographic_info"]["issue"] if "issue" in entry["bibliographic_info"].keys() else ""
    work["first_page"] = entry["bibliographic_info"]["first_page"] if "first_page" in entry["bibliographic_info"].keys() else ""
    work["last_page"] = entry["bibliographic_info"]["last_page"] if "last_page" in entry["bibliographic_info"].keys() else ""
    authors = []
    for author in entry['authors']:
        if len(authors) >= 5:
            break
        if "full_name" in author.keys():
            authors.append(author["full_name"])
    work["authors"] = authors
    es_handler.insert_work(_id=str(entry["_id"]), work=work)

In [7]:
#[Parallel(n_jobs=20)]: Done 258748 tasks      | elapsed: 13.9min
#[Parallel(n_jobs=20)]: Done 260144 tasks      | elapsed: 13.9min
#[Parallel(n_jobs=20)]: Done 261540 tasks      | elapsed: 14.0min
#[Parallel(n_jobs=20)]: Done 262944 tasks      | elapsed: 14.1min

res = Parallel(n_jobs=20,backend="threading",verbose=10)(delayed(insert_work)(entry) for entry in kahi_data)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  21 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  32 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  45 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  73 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done  88 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 105 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 141 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 160 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 202 tasks      | elapsed:    0.2s
[Parallel(n_jobs=20)]: Done 225 tasks      | elaps

In [12]:
empty=[i for i in res if i != None ] #articulos sin titulo, no deben ser considerados.

In [13]:
len(empty)

38

In [14]:
empty[0]

{'_id': ObjectId('657768c33fae4cca97bd55be'),
 'titles': [],
 'updated': [{'source': 'openalex', 'time': 1702324419}],
 'subtitle': '',
 'abstract': '',
 'keywords': [],
 'types': [{'source': 'openalex', 'type': 'journal-issue'}],
 'external_ids': [{'source': 'openalex',
   'id': 'https://openalex.org/W3206547180'},
  {'source': 'doi', 'id': '10.24245/dermatolrevmex.v65i5.6902'},
  {'source': 'mag', 'id': '3206547180'}],
 'external_urls': [],
 'date_published': 1633928400,
 'year_published': 2021,
 'bibliographic_info': {'volume': '65',
  'issue': '5',
  'is_open_access': False,
  'open_access_status': 'closed'},
 'references_count': None,
 'references': [],
 'citations_count': [{'source': 'openalex', 'count': 0}],
 'citations': [],
 'author_count': 1,
 'authors': [{'id': ObjectId('657748733fae4cca97ab583e'),
   'full_name': 'Álvaro Monterrosa-Castro',
   'affiliations': [{'id': ObjectId('65773ee73fae4cca97a47b1e'),
     'name': 'University of Cartagena',
     'types': [{'source': 'ror